# NPM3D PanopticSeg — env stable (Py3.10) + embeddings + éval
Ce notebook évite le piège Python 3.12 / Torch 2.x en créant un **environnement Python 3.10** isolé via **micromamba**.
On installe les **versions récentes compatibles** avec MinkowskiEngine, puis on suit le pipeline officiel.

**Stack choisie (stable et la plus récente compatible ME):**
- Python 3.10 (environnement micromamba `npme`)
- PyTorch **1.13.1 + cu116**
- torch-geometric **2.3.1** (+ roues scatter/sparse/cluster/spline-conv pour 1.13.1+cu116)
- MinkowskiEngine **0.5.4** (roues précompilées NVIDIA)

Tout ce qui tourne côté Python dans la suite passe par `micromamba run -n npme` pour être sûr qu’on reste dans le bon env.

## 0) Vérif GPU

In [ ]:

!nvidia-smi || true


## 1) Installer micromamba, créer env Py3.10 et installer les libs compatibles ME

In [ ]:

    %%bash
    set -euxo pipefail

    export MAMBA_ROOT_PREFIX=/content/micromamba
    if [ ! -d "$MAMBA_ROOT_PREFIX" ]; then
      curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
      install -Dm755 bin/micromamba /usr/local/bin/micromamba
    fi

    # Créer l'env si nécessaire
    if ! micromamba env list | grep -q "^npme"; then
      micromamba create -y -n npme -c conda-forge python=3.10 pip
    fi

    # Installer la stack dans l'env
    micromamba run -n npme python -m pip install --upgrade pip wheel setuptools packaging

    # Torch 1.13.1 + cu116
    micromamba run -n npme python -m pip install --no-cache-dir \
      torch==1.13.1+cu116 torchvision==0.14.1+cu116 \
      -f https://download.pytorch.org/whl/torch_stable.html

    # PyG wheels pour torch-1.13.1+cu116
    micromamba run -n npme python -m pip install --no-cache-dir \
      --extra-index-url https://data.pyg.org/whl/torch-1.13.1+cu116.html \
      torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric==2.3.1

    # MinkowskiEngine roues (évite toute compilation CUDA)
    micromamba run -n npme python -m pip install --no-cache-dir \
      MinkowskiEngine==0.5.4 -f https://nvidia.github.io/MinkowskiEngine/

    # Dépendances repo
    micromamba run -n npme python -m pip install --no-cache-dir \
      hydra-core==1.1.0 omegaconf==2.1.0 plyfile==0.8.1 \
      scipy==1.10.1 hdbscan==0.8.29 pandas==1.5.3 numba==0.57.1 joblib==1.3.2 tqdm pyyaml

    # Sanity check
    micromamba run -n npme python - <<'PY'
import torch, sys
print("torch:", torch.__version__, "cuda:", torch.version.cuda, "CUDA available:", torch.cuda.is_available())
try:
    import MinkowskiEngine as ME
    print("MinkowskiEngine:", ME.__version__)
except Exception as e:
    print("ME import failed:", e); sys.exit(1)
try:
    import torch_geometric
    print("torch_geometric:", torch_geometric.__version__)
except Exception as e:
    print("pyg import failed:", e); sys.exit(1)
PY


## 2) Cloner ton fork du repo

In [ ]:

%%bash
set -euxo pipefail

REPO_URL="https://github.com/Ludwig-H/PanopticSegForLargeScalePointCloud"
if [ ! -d PanopticSegForLargeScalePointCloud ]; then
  git clone --depth=1 "$REPO_URL"
fi
cd PanopticSegForLargeScalePointCloud
mkdir -p data/npm3dfused/raw outputs
echo "Repo ok."


## 3) Télécharger NPM3D (labels d’instances) dans `data/npm3dfused/raw`

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud/data/npm3dfused/raw

base="https://zenodo.org/records/8118986/files"
files=(
  "Paris_train.ply" "Paris_val.ply" "Paris_test.ply"
  "Lille1_1_train.ply" "Lille1_1_val.ply" "Lille1_1_test.ply"
  "Lille1_2_train.ply" "Lille1_2_val.ply" "Lille1_2_test.ply"
  "Lille2_train.ply" "Lille2_val.ply" "Lille2_test.ply"
)
for f in "${files[@]}"; do
  if [ ! -f "$f" ]; then
    echo "Downloading $f"
    wget -q --show-progress "${base}/${f}?download=1" -O "$f"
  else
    echo "Already have $f"
  fi
done
ls -lh


## 4) (Optionnel) Entraînement Setting IV (rayon 16 m, voxel 0.12)

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud
export WANDB_MODE=offline

echo "Exemple de commande (désactivée par défaut) :"
echo "micromamba run -n npme python train.py task=panoptic \\
  data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1 \\
  models=panoptic/area4_ablation_3heads_5 model_name=PointGroup-PAPER \\
  training=7_area1 job_name=A1_S7"


## 5) Adapter `conf/eval.yaml` et préparer un checkpoint
Si tu n’entraînes pas dans cette session, monte ton Drive et pointe `CKPT_PATH` sur un checkpoint `best.pth` existant.
Le fichier `conf/eval.yaml` contrôle chemins et split.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud
echo "Aperçu conf/eval.yaml (éditable si besoin):"
sed -n '1,200p' conf/eval.yaml || true


## 6) Extraction **embeddings 5D** avant clustering
On ajoute un script indépendant qui charge le modèle/dataloader d’éval via Hydra, accroche un hook sur la tête d’embedding et dump des `.npz`.

In [ ]:

    %%bash
    set -euxo pipefail
    cd PanopticSegForLargeScalePointCloud

    cat > tools_extract_embeddings.py << 'PY'
import os, re, numpy as np, torch
from pathlib import Path
import hydra
from omegaconf import DictConfig, OmegaConf

# Stratégie: réutiliser la construction des loaders & du modèle d'eval.
# Le repo expose généralement des helpers dans eval.py; sinon, on importe train/eval et
# cherche les objets dans le namespace hydra.

def _attach_first_embed_hook(model, bucket):
    chosen = None
    for name, m in model.named_modules():
        if re.search(r"(emb|embed|embedding)", name, re.I):
            chosen = (name, m)
            break
    if chosen is None:
        print("[extract] Aucun module 'emb*' trouvé; on essaiera de lire la sortie du forward dict.")
        return None
    name, mod = chosen
    print(f"[extract] Hook sur '{name}'")
    def _hook(module, inp, out):
        try:
            E = out.detach().float().cpu().numpy()
            bucket.append(E)
        except Exception as e:
            print("[extract] hook fail:", e)
    h = mod.register_forward_hook(lambda m,i,o: _hook(m,i,o))
    return h

@hydra.main(config_path="conf", config_name="eval", version_base=None)
def main(cfg: DictConfig):
    # On s'appuie sur les mêmes builders que eval.py
    # Les fonctions peuvent s'appeler différemment selon commit; on essaye plusieurs options.
    build_ok = False
    model = None; loaders = None; device = None; test_split = None
    try:
        from eval import build_model_and_loaders
        model, loaders, device, test_split = build_model_and_loaders(cfg)
        build_ok = True
        print("[extract] build_model_and_loaders OK")
    except Exception as e:
        print("[extract] Pas de build_model_and_loaders dans eval.py:", e)

    if not build_ok:
        try:
            # Certains repos exposent make_model, make_dataloaders
            from eval import make_model, make_dataloaders, get_device
            device = get_device(cfg)
            model = make_model(cfg).to(device).eval()
            loaders, test_split = make_dataloaders(cfg)
            build_ok = True
            print("[extract] make_model/make_dataloaders OK")
        except Exception as e:
            print("[extract] Echec construction modèle/loaders:", e)
            raise SystemExit(1)

    out_root = Path("outputs/embeddings")
    out_root.mkdir(parents=True, exist_ok=True)

    bucket = []
    hook = _attach_first_embed_hook(model, bucket)

    test_loader = loaders.get(test_split, None) or loaders.get("test", None)
    if test_loader is None:
        print("[extract] test_loader introuvable. Clés:", list(loaders.keys()))
        raise SystemExit(1)

    with torch.no_grad():
        for ib, batch in enumerate(test_loader):
            batch = batch.to(device)
            out = model(batch)
            # Si le modèle retourne un dict avec une clé embeddings*
            if isinstance(out, dict):
                for k in ["embedding", "embeddings", "embedding_5d", "emb_5d", "panoptic_embeddings"]:
                    if k in out and isinstance(out[k], torch.Tensor):
                        E = out[k].detach().cpu().float().numpy()
                        bucket.append(E)
                        break
            if not bucket:
                print(f"[extract] WARNING: pas d'embeddings capturés pour batch {ib}.")
                continue
            E = bucket[-1]
            idx = getattr(batch, "idx", None)
            if idx is None and hasattr(batch, "ptr"):
                idx = batch.ptr
            if idx is None:
                idx = torch.arange(E.shape[0], device=E.device)
            idx = idx.detach().cpu().numpy()
            tag = f"batch_{ib:05d}"
            np.savez_compressed(out_root / f"{tag}.npz", embeddings=E, indices=idx)
            print(f"[extract] saved {tag}: {E.shape}")
    if hook is not None:
        hook.remove()

if __name__ == "__main__":
    main()
PY

    echo "tools_extract_embeddings.py créé."


## 7) Lancer éval officielle puis extraction d’embeddings
Adapte `CKPT_PATH` si besoin. Tu peux aussi surcharger la conf Hydra en CLI.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud

# Exemple: pointer vers un checkpoint existant (modifie ce chemin selon ton stockage)
export CKPT_PATH="${CKPT_PATH:-/content/drive/MyDrive/panoptic_runs/A1_S7/checkpoints/best.pth}"

echo "Si nécessaire, lance l'éval (désactivée ici si conf déjà prête):"
echo "micromamba run -n npme python eval.py"

echo "Extraction embeddings:"
micromamba run -n npme python tools_extract_embeddings.py || true
ls -lh outputs/embeddings || true


## 8) Brancher ton clusterer sur les embeddings `.npz`
Cette cellule de **démonstration** lit tous les `.npz`, normalise, applique un clusterer bidon et sauve un `instance_labels.npy`.

In [ ]:

    %%bash
    set -euxo pipefail
    cd PanopticSegForLargeScalePointCloud

    cat > tools_cluster_with_embeddings.py << 'PY'
import os, glob, numpy as np
from sklearn.preprocessing import StandardScaler

emb_dir = "outputs/embeddings"
out_path = "outputs/instance_labels.npy"

def load_all_npz(d):
    Xs, Is = [], []
    for f in sorted(glob.glob(os.path.join(d, "*.npz"))):
        d = np.load(f)
        Xs.append(d["embeddings"])
        Is.append(d["indices"])
    X = np.concatenate(Xs, axis=0) if Xs else np.zeros((0,5), dtype=np.float32)
    I = np.concatenate(Is, axis=0) if Is else np.zeros((0,), dtype=np.int64)
    return X, I

def my_clusterer(X):
    # TODO: remplace par ton algo
    return np.zeros(X.shape[0], dtype=np.int32)

X, I = load_all_npz(emb_dir)
if X.shape[0] == 0:
    raise SystemExit("Aucun embedding trouvé. Vérifie l'étape précédente.")
Xn = StandardScaler().fit_transform(X)
labels = my_clusterer(Xn)
np.save(out_path, labels)
print("Saved:", out_path, labels.shape)
PY

    micromamba run -n npme python -m pip install scikit-learn || true
    micromamba run -n npme python tools_cluster_with_embeddings.py


## 9) Évaluation finale (scripts officiels)

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud
micromamba run -n npme python evaluation_stats_NPM3D.py || true


## Notes
- Si tu tiens absolument à rester en Python 3.12/Torch 2.x natif, **MinkowskiEngine** risque de ne pas avoir de roue binaire. Ne compile pas en Colab, c’est peine perdue sans toolkit CUDA complet. L’env micromamba **résout le problème proprement**.
- Tu peux dupliquer l’éval sur `area{1..4}` (4-fold) en adaptant la conf Hydra comme dans le README.
- Si la capture d’embeddings ne prend pas, imprime `model.named_modules()` et ajuste le motif de recherche `'emb'` dans `tools_extract_embeddings.py`.